In [ ]:
'''
This is the outline of an approach.
A real effort would break down each time slice by day (or finer).
The vol for each time slice would be amplified or dampened by characteristics of the day (weekend, holiday, important news, etc.)
The output would be the forward vol and spot vol to each date such that the they spot vol agreed to any listed vol expiry. 

In [47]:
import xlwings as xw
import pandas as pd
import matplotlib.pyplot as plt

from datetime import timedelta

from Options_Math_Helpers import *
from Options_Math_Algebra import *
from Options_Math_Black_Scholes import *
from Options_Wow_Alpha_Skew import *

omh  = OptionsMathHelpers()
oma  = OptionsMathAlgebra()
ombs = OptionsMathBlackScholes()
wa   = WowAlphaSkew()

In [48]:
# Open workbook
wb = xw.Book('deribit.xlsx')

# Select sheet by name
sheet = wb.sheets['wow_alpha_params']

# Read entire used range into a DataFrame
df = sheet.used_range.options(pd.DataFrame, header=True, index=False).value

print(df)

              exp_time  yrs_to_exp   atm_vol       wow      conv  model_conv
0  2025-11-19 03:00:00    0.000459  0.503872  0.319049 -9.274169   -9.269282
1  2025-11-20 03:00:00    0.003197  0.527534  0.351259 -4.203926   -4.249758
2  2025-11-21 03:00:00    0.005935  0.536214  0.324578 -3.124340   -3.015657
3  2025-11-22 03:00:00    0.008673  0.527924  0.301441 -2.599905   -2.605650
4  2025-11-28 03:00:00    0.025100  0.484476  0.332550 -1.555966   -1.678057
5  2025-12-05 03:00:00    0.044265  0.480141  0.356317 -1.062043   -1.072725
6  2025-12-26 03:00:00    0.101760  0.483629  0.349632 -0.632048   -0.413348
7  2026-01-30 03:00:00    0.197584  0.483546  0.343537 -0.394847   -0.270036
8  2026-03-27 04:00:00    0.350904  0.487509  0.362076 -0.257426   -0.260715
9  2026-06-26 04:00:00    0.600049  0.490323  0.361648 -0.155300   -0.260605
10 2026-09-25 04:00:00    0.849193  0.491819  0.366850 -0.096468   -0.260605


In [49]:
df = df.drop(columns=['wow', 'conv', 'model_conv'])
df = df.rename(columns={'exp_time' : 'end'})
df.insert(0, 'start', df['end'].shift(1))

df.loc[0, 'start'] = df.loc[0, 'end'] - timedelta(days = df.loc[0, 'yrs_to_exp'] * 365.25)

df['end - start'] =  df['yrs_to_exp'].diff()
df.loc[0, 'end - start'] = df.loc[0, 'yrs_to_exp']

In [54]:
df['targets'] = df['atm_vol'] * df['atm_vol'] * df['yrs_to_exp']

df['fwd_targets'] = df['targets'].diff()
df.loc[0, 'fwd_targets'] = df.loc[0, 'targets']

df['fwd_vol'] = df['fwd_targets'] / df['end - start']
df['fwd_vol'] = np.sqrt(df['fwd_vol'])



print(df)

                        start                 end  yrs_to_exp   atm_vol  \
0  2025-11-18 22:58:28.094855 2025-11-19 03:00:00    0.000459  0.503872   
1  2025-11-19 03:00:00.000000 2025-11-20 03:00:00    0.003197  0.527534   
2  2025-11-20 03:00:00.000000 2025-11-21 03:00:00    0.005935  0.536214   
3  2025-11-21 03:00:00.000000 2025-11-22 03:00:00    0.008673  0.527924   
4  2025-11-22 03:00:00.000000 2025-11-28 03:00:00    0.025100  0.484476   
5  2025-11-28 03:00:00.000000 2025-12-05 03:00:00    0.044265  0.480141   
6  2025-12-05 03:00:00.000000 2025-12-26 03:00:00    0.101760  0.483629   
7  2025-12-26 03:00:00.000000 2026-01-30 03:00:00    0.197584  0.483546   
8  2026-01-30 03:00:00.000000 2026-03-27 04:00:00    0.350904  0.487509   
9  2026-03-27 04:00:00.000000 2026-06-26 04:00:00    0.600049  0.490323   
10 2026-06-26 04:00:00.000000 2026-09-25 04:00:00    0.849193  0.491819   

    end - start   targets  fwd_targets   fwd_vol  
0      0.000459  0.000117     0.000117  0.503872

In [56]:
sheet = wb.sheets('daily_vol_calc')
sheet.range('a1').options(index = False, header=1).value = df